In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
from UtilsBC import *
from DatasetBC import *
torch.cuda.set_device(1)

In [ ]:
for k in tqdm_notebook(datasets) :
    print(k)
    dataset = datasets[k]()
    dataset.display_stats()
#     test_data = dataset.test_data
#     model = load_model(dataset)
#     test_data.yt_hat, test_data.attn_hat = evaluate_and_print(model, test_data)
#     generate_graphs(dataset, model, test_data)

In [ ]:
for k in tqdm_notebook(datasets) :
    wd = [0, 1e-5, 1e-4, 1e-3]
    print(k)
    if k == 'sst' :
        dataset = datasets[k]()
        for w in wd :
            dataset.weight_decay = w
            train(dataset, name='lambda=' + str(w), exp_dirname='exp_lambda')

**Training**
============

In [ ]:
dataset = AGNews_dataset()

In [ ]:
train(dataset, name='test')

In [ ]:
dataset.set_model('outputs/attn_word_agnews/SatNov1714:18:152018_test/')

# **EVALUATION**

In [ ]:
model = load_model(dataset)
test_data = dataset.test_data
test_data.yt_hat, test_data.attn_hat = evaluate_and_print(model, test_data)

**Generate Graphs**
===================

In [ ]:
generate_graphs(dataset, model, test_data)

# __Sampling__

In [ ]:
model.vec = dataset.vec
sampled_output = model.sampling_top(test_data.X, sample_vocab=100, topnum=5)
pdump(model, sampled_output, 'sampled')

**Permutation**
===========

In [ ]:
perms = model.permute_attn(test_data.X)
pdump(model, perms, 'permutations')

**Multi Adversarial Attention**
===============================

In [ ]:
multi_adversarial_outputs = model.adversarial_multi(test_data.X, _type='uniform')
pdump(model, multi_adversarial_outputs, 'multi_adversarial')

In [ ]:
multi_adversarial_outputs = pload(model, 'multi_adversarial')

In [ ]:
emax_jds, emax_adv_attn, emax_ad_y = plot_multi_adversarial(test_data.X, test_data.yt_hat, test_data.attn_hat, 
                                                            multi_adversarial_outputs, dirname=model.dirname)

In [ ]:
np.argsort(emax_jds)[-30:]

In [ ]:
n = 2932
print_adversarial_example(dataset.vec.map2words(test_data.X[n]), test_data.attn_hat[n], emax_adv_attn[n])
print(test_data.yt_hat[n], emax_ad_y[n])

**Remove and Run** -- p(y|x, c) - p(y|c)
==================

In [ ]:
remove_outputs = model.remove_and_run(test_data.X)
pdump(model, remove_outputs, 'remove_and_run')